In [1]:
%load_ext autoreload
%autoreload 2

%config IPython.sys.argv=['-Xfrozen_modules=off']

from qiskit_metal import draw, Dict, designs, MetalGUI
from qiskit_metal.toolbox_metal import math_and_overrides
from qiskit_metal.qlibrary.core import QComponent
import qiskit_metal as metal
# ?metal.qlibrary.core.QComponent.add_qgeometry

design = metal.designs.design_planar.DesignPlanar()
gui = metal.MetalGUI(design)
design.overwrite_enabled = True

# dir(QComponent)
cpw_width = '11.7um'

design._chips.main.size.size_x = '5mm'
design._chips.main.size.size_y = '5mm'
from qiskit_metal.qlibrary.terminations.launchpad_wb import LaunchpadWirebond
from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.tlines.anchored_path import RouteAnchors
from qiskit_metal.qlibrary.tlines.mixed_path import RouteMixed
from qiskit_metal.qlibrary.qubits.transmon_cross import TransmonCross
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.couplers.cap_n_interdigital_tee import CapNInterdigitalTee
from qiskit_metal.qlibrary.couplers.line_tee import LineTee

from collections import OrderedDict

import numpy as np

In [2]:
xmon_options = dict(
    connection_pads=dict(
        c = dict(
                 claw_length = '460um',
                 ground_spacing = '3um', 
                 claw_gap = '5.1um', 
                 claw_width = '15um',
                 claw_cpw_length = '50um'),
    ),
    cross_width = '30um',
    cross_length = '500um',
    cross_gap = '30um',
    aedt_hfss_inductance = 12E-9,

)

# Create a new Transmon Cross object with name 'Q1'
q = TransmonCross(design, 'Q', options=xmon_options)

gui.rebuild()  # line needed to propagate the updates from the qubit instance into the junction design table
gui.autoscale()

# LOM sim

In [3]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")

In [4]:
c1.sim.setup.reuse_selected_design = False
c1.sim.setup.reuse_setup = False

# example: update single setting
c1.sim.setup.max_passes = 30
c1.sim.setup.min_converged_passes = 1
c1.sim.setup.percent_error = 0.1
c1.sim.setup.name = 'sweep_setup'

In [47]:
c1.sim.renderer.render_design(['Q'], [('Q', 'c')])

05:39PM 35s ERROR [subtract_from_ground]: This error might indicate that a component was not correctly rendered in Ansys. 
This might have been caused by floating point numerical corrections. 
 For example Ansys will inconsistently render (or not) routing that has 180deg jogs with the two adjacent segments spaced 'exactly' twice the fillet radius (U shaped routing). 
In this example, changing your fillet radius to a smaller number would solve the issue.


com_error:  (-2147352567, 'Exception occurred.', (0, None, None, None, 0, -2147024381), None)


com_error: (-2147352567, 'Exception occurred.', (0, None, None, None, 0, -2147024381), None)

In [38]:
for i in range(2):
    components = ['Q']
    open_terminations = [('Q', 'c')]

    # if components is not None:
    #     argm = dict(locals())
    #     del argm['self']
    #     self.save_run_args(**argm)
    # wipe data from the previous run (if any)
    c1.sim.clear_data()

    if not c1.sim.renderer_initialized:
        c1.sim._initialize_renderer()

    renderer_design_name = c1.sim._render(name='test_sim_Q',
                                        solution_type='capacitive',
                                        selection=components,
                                        open_pins=open_terminations,
                                        box_plus_buffer=True,
                                        vars_to_initialize=Dict())

    c1.sim.renderer.analyze_setup(c1.sim.setup.name)

    cap_df = c1.sim.capacitance_matrix

    print(cap_df)



INFO 05:31PM [connect_design]: 	Opened active design
	Design:    test_sim_Q_q3d [Solution type: Q3D]
WARNING 05:31PM [connect_setup]: 	No design setup detected.
WARNING 05:31PM [connect_setup]: 	Creating Q3D default setup.
INFO 05:31PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)


OSError: Setup sweep_setup not found: ('Setup',)

In [5]:
def render_design():
    c1.sim.renderer.render_design()

In [6]:
def run_simulation():
    c1.sim.setup.reuse_selected_design = False
    c1.sim.setup.reuse_setup = False

    # example: update single setting
    c1.sim.setup.max_passes = 30
    c1.sim.setup.min_converged_passes = 1
    c1.sim.setup.percent_error = 0.1
    c1.sim.setup.name = 'sweep_setup'

    c1.sim.add_setup(name = c1.sim.setup.name)
    
    Lj = design.components['Q'].options['aedt_hfss_inductance']
    design.components['Q'].options['hfss_inductance'] = Lj
    design.rebuild()
    
    c1.sim.analyze_setup(name = c1.sim.setup.name)

def add_project_level_vars(renderer):
    Lj = design.components['Q'].options['aedt_hfss_inductance']
    design.components['Q'].options['hfss_inductance'] = Lj
    design.rebuild()

def run_analysis():
    cap_df = c1.sim.capacitance_matrix
    cap_df['misc'] = [ground_spacing_arr[i], cross_len_arr[j], claw_len_arr[k]]
    return cap_df

In [8]:
import pandas as pd

ground_spacing_arr = [3, 22, 30]
cross_len_arr = [50+i*50 for i in range(10)]
claw_len_arr = [10+i*50 for i in range(10)]

all_cap_matricies = []

# c1.sim.renderer.activate_ansys_setup(name = 'sweep_setup')


for i in ground_spacing_arr:
    for j in range(len(cross_len_arr)):
        for k in range(j+1):
            if i == 22 and cross_len_arr[j] <= 300 and claw_len_arr[k] <= 260:
                continue
            design.components['Q'].options['connection_pads']['c']['ground_spacing'] = str(i) + 'um'
            design.components['Q'].options['cross_length'] = str(cross_len_arr[j]) + 'um'
            design.components['Q'].options['connection_pads']['c']['claw_length'] = str(claw_len_arr[k]) + 'um'
            
            Lj = design.components['Q'].options['aedt_hfss_inductance']
            design.components['Q'].options['hfss_inductance'] = Lj
            design.rebuild()

            # c1.sim.renderer.render_design(selection=['Q'], open_pins=[('Q', 'c')])
            # c1.sim.analyze_setup(name = 'sweep_setup')

            c1.sim.run(name = 'sweep_v4.0', components=['Q'],
               open_terminations=[('Q', 'c')])

            cap_df = c1.sim.capacitance_matrix
            cap_df['misc'] = [i, cross_len_arr[j], claw_len_arr[k]]
            all_cap_matricies.append(cap_df)
            cap_df.to_csv('cap_matrices_08_23_3GS_redo.csv', mode = 'a', index = True, header = False)

            print(f'For Qubit_{i}_{cross_len_arr[j]}_{claw_len_arr[k]}, the cap matrix is...')
            print(cap_df)

final_df = pd.concat(all_cap_matricies)

INFO 04:00PM [connect_design]: 	Opened active design
	Design:    sweep_v4.0_q3d1 [Solution type: Q3D]
WARNING 04:00PM [connect_setup]: 	No design setup detected.
WARNING 04:00PM [connect_setup]: 	Creating Q3D default setup.
INFO 04:00PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 04:00PM [get_setup]: 	Opened setup `sweep_setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 04:00PM [analyze]: Analyzing setup sweep_setup
INFO 04:02PM [get_matrix]: Exporting matrix data to (C:\Users\lfl\AppData\Local\Temp\tmp3vbayox1.txt, C, , sweep_setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 1, False
INFO 04:02PM [get_matrix]: Exporting matrix data to (C:\Users\lfl\AppData\Local\Temp\tmp8jmpgfm0.txt, C, , sweep_setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 1, False
INFO 04:02PM [get_matrix]: Exporting matrix data to (C:\Users\lfl\AppData\Local\Temp\tmp06kfmw6y.txt, C, , sweep_setup:AdaptivePass, "Origin

For Qubit_3_50_10, the cap matrix is...
                   c_connector_arm_Q   cross_Q  ground_main_plane  misc
c_connector_arm_Q           38.09102  -1.29132          -36.49523     3
cross_Q                     -1.29132  25.66381          -23.62833    50
ground_main_plane          -36.49523 -23.62833           97.26806    10


INFO 04:02PM [connect_design]: 	Opened active design
	Design:    sweep_v4.0_q3d2 [Solution type: Q3D]
WARNING 04:02PM [connect_setup]: 	No design setup detected.
WARNING 04:02PM [connect_setup]: 	Creating Q3D default setup.
INFO 04:02PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 04:02PM [get_setup]: 	Opened setup `sweep_setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 04:02PM [analyze]: Analyzing setup sweep_setup
INFO 04:04PM [get_matrix]: Exporting matrix data to (C:\Users\lfl\AppData\Local\Temp\tmpgbx9z6z6.txt, C, , sweep_setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 1, False
INFO 04:04PM [get_matrix]: Exporting matrix data to (C:\Users\lfl\AppData\Local\Temp\tmpopdp6rnx.txt, C, , sweep_setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 1, False
INFO 04:04PM [get_matrix]: Exporting matrix data to (C:\Users\lfl\AppData\Local\Temp\tmp1d3k388p.txt, C, , sweep_setup:AdaptivePass, "Origin

For Qubit_3_100_10, the cap matrix is...
                   c_connector_arm_Q   cross_Q  ground_main_plane  misc
c_connector_arm_Q           38.06734  -1.24441          -36.55934     3
cross_Q                     -1.24441  50.35685          -47.69446   100
ground_main_plane          -36.55934 -47.69446          126.19082    10


INFO 04:04PM [connect_design]: 	Opened active design
	Design:    sweep_v4.0_q3d3 [Solution type: Q3D]
WARNING 04:04PM [connect_setup]: 	No design setup detected.
WARNING 04:04PM [connect_setup]: 	Creating Q3D default setup.
INFO 04:04PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 04:04PM [get_setup]: 	Opened setup `sweep_setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 04:04PM [analyze]: Analyzing setup sweep_setup
INFO 04:05PM [get_matrix]: Exporting matrix data to (C:\Users\lfl\AppData\Local\Temp\tmpvtyv9bje.txt, C, , sweep_setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 1, False
INFO 04:05PM [get_matrix]: Exporting matrix data to (C:\Users\lfl\AppData\Local\Temp\tmpi0pfvx_9.txt, C, , sweep_setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 1, False
INFO 04:05PM [get_matrix]: Exporting matrix data to (C:\Users\lfl\AppData\Local\Temp\tmp26pkwaxi.txt, C, , sweep_setup:AdaptivePass, "Origin

...

For Qubit_30_500_360, the cap matrix is...
                   c_connector_arm_Q    cross_Q  ground_main_plane  misc
c_connector_arm_Q          181.75503   -4.20004         -177.05802    30
cross_Q                     -4.20004  271.29557         -264.42314   500
ground_main_plane         -177.05802 -264.42314          520.96920   360


INFO 11:46PM [connect_design]: 	Opened active design
	Design:    sweep_v4.0_q3d143 [Solution type: Q3D]
WARNING 11:46PM [connect_setup]: 	No design setup detected.
WARNING 11:46PM [connect_setup]: 	Creating Q3D default setup.
INFO 11:46PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 11:46PM [get_setup]: 	Opened setup `sweep_setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 11:46PM [analyze]: Analyzing setup sweep_setup
INFO 11:49PM [get_matrix]: Exporting matrix data to (C:\Users\lfl\AppData\Local\Temp\tmpdzf4jsrz.txt, C, , sweep_setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 1, False
INFO 11:49PM [get_matrix]: Exporting matrix data to (C:\Users\lfl\AppData\Local\Temp\tmpneaz1adx.txt, C, , sweep_setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 1, False
INFO 11:49PM [get_matrix]: Exporting matrix data to (C:\Users\lfl\AppData\Local\Temp\tmp2osrjum2.txt, C, , sweep_setup:AdaptivePass, "Orig

For Qubit_30_500_410, the cap matrix is...
                   c_connector_arm_Q    cross_Q  ground_main_plane  misc
c_connector_arm_Q          199.63894   -4.91716         -194.16881    30
cross_Q                     -4.91716  270.46999         -262.87110   500
ground_main_plane         -194.16881 -262.87110          536.88777   410


INFO 11:50PM [connect_design]: 	Opened active design
	Design:    sweep_v4.0_q3d144 [Solution type: Q3D]
WARNING 11:50PM [connect_setup]: 	No design setup detected.
WARNING 11:50PM [connect_setup]: 	Creating Q3D default setup.
INFO 11:50PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 11:50PM [get_setup]: 	Opened setup `sweep_setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 11:50PM [analyze]: Analyzing setup sweep_setup
INFO 11:55PM [get_matrix]: Exporting matrix data to (C:\Users\lfl\AppData\Local\Temp\tmpc8393hkp.txt, C, , sweep_setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 1, False
INFO 11:55PM [get_matrix]: Exporting matrix data to (C:\Users\lfl\AppData\Local\Temp\tmp40a6k965.txt, C, , sweep_setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 1, False
INFO 11:55PM [get_matrix]: Exporting matrix data to (C:\Users\lfl\AppData\Local\Temp\tmp1mtralnz.txt, C, , sweep_setup:AdaptivePass, "Orig

For Qubit_30_500_460, the cap matrix is...
                   c_connector_arm_Q    cross_Q  ground_main_plane  misc
c_connector_arm_Q          219.43827   -5.79075         -213.04679    30
cross_Q                     -5.79075  271.16112         -262.68817   500
ground_main_plane         -213.04679 -262.68817          555.27539   460


In [49]:
final_df = pd.concat(all_cap_matricies)
final_df.to_csv('cap_matrices_08_23.csv')

In [63]:
for i in ground_spacing_arr[-2:]:
    print(i)

22
30


In [61]:
cap_df.to_csv('cap_matrices_08_22_copy.csv', mode = 'a', index = True, header = True)

In [81]:
from qiskit_metal.renderers.renderer_ansys_pyaedt.q3d_renderer_aedt import QQ3DPyaedt

project_name = c1.sim.renderer.pinfo.project_name
design_name = c1.sim.renderer.get_active_design_name()
# c1.sim.renderer.render_design()

# c1.sim.setup.reuse_selected_design = False
# c1.sim.setup.reuse_setup = False

# # example: update single setting
# c1.sim.setup.max_passes = 30
# c1.sim.setup.min_converged_passes = 1
# c1.sim.setup.percent_error = 0.1
# c1.sim.setup.name = 'sweep_setup'

# # c1.sim.add_setup(name = c1.sim.setup.name)

# qq3d = QQ3DPyaedt(design)
# qq3d.add_q3d_setup()  

# c1.sim.start()
dir(c1.sim.renderer.pinfo.setup)
# c1.sim.setup
# c1.sim.renderer.setup

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_ansys_version',
 '_readin_Q3D_matrix',
 '_reporter',
 '_setup_module',
 '_solutions',
 'add_fields_convergence_expr',
 'analyze',
 'basis_order',
 'commit_convergence_exprs',
 'delete_sweep',
 'delta_f',
 'expression_cache_items',
 'frequency',
 'get_convergence',
 'get_fields',
 'get_frequency_Hz',
 'get_matrix',
 'get_mesh_stats',
 'get_profile',
 'get_solutions',
 'get_sweep',
 'get_sweep_names',
 'insert_sweep',
 'load_q3d_matrix',
 'max_pass',
 'min_freq',
 'min_pass',
 'n_modes',
 'name',
 'parent',
 'passes',
 'pct_error',
 'pct_refinement',
 'prop_holder',
 'prop_server',
 'prop_tab',
 'release'

In [11]:
def run_single_design(): # this will be used as `custom_analysis`

    render_design()
    run_simulation()
    package = run_analysis()
    return package

In [12]:
from metal_library.sweeper import QSweeper
sweeper = QSweeper(design)

In [13]:
sweep_parameters = dict(
  connection_pads = dict(
    readout = dict(
      connector_type = '0', 
      connector_location = '0',
      claw_length = [f'{10+i*50}um' for i in range(10)],
      ground_spacing = ['3um', '4um', '5um', '7um', '9um', '15um', '22um', '30um'], 
      claw_gap = ['5.1um'], 
      claw_width = ['15um'],
      claw_cpw_length = ['50um'],
      claw_cpw_width = ['10um']
      )
    ),
  cross_width = ['30um'],
  cross_length = [f'{50+i*50}um' for i in range(10)],
  cross_gap = ['30um'],
  aedt_hfss_inductance = [12E-9]
)
my_librarian = sweeper.run_single_component_sweep(component_name='Q',
                                                  parameters = sweep_parameters,
                                                  custom_analysis = run_single_design,
                                                  parameters_slice=slice(1,2),
                                                  save_path="./sweep_08_19_test.csv")

  0%|          | 0/1 [00:00<?, ?it/s]


AttributeError: 'NoneType' object has no attribute 'draw_polyline'

In [13]:
import os
import qiskit_metal as metal 
module_path = os.path.abspath(metal.__file__)
print("Path:", module_path)

Path: c:\users\lfl\qiskit-metal\qiskit_metal\__init__.py
